In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [69]:
# input
pvar_f   <- '/oak/stanford/groups/mrivas/ukbb24983/cal/pgen/ukb24983_cal_cALL_v2_hg19.pvar'
csq_rules_f <- 'VEP_consequence_group.annotated.tsv'
vep_f    <- '/scratch/groups/mrivas/ukbb24983/cal/annotation_20201002/ukb24983_cal_cALL_v2_hg19.vep101-loftee.tsv.gz'
# output
vep_csq_f <- '/scratch/groups/mrivas/ukbb24983/cal/annotation_20201002/ukb24983_cal_cALL_v2_hg19.vep101-loftee.Csq.tsv'


In [10]:
pvar_f %>% fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') %>% mutate(pvar_order = 1:n()) -> pvar_df


In [19]:
csq_rules_f %>% fread(select=c('#Consequence', 'Csq')) %>%
rename('Consequence'='#Consequence') -> csq_rules_df


In [4]:
vep_f %>% fread(colClasses = c('#CHROM'='character')) %>% rename('CHROM'='#CHROM') -> vep_df


In [66]:
pvar_df %>%
mutate(CHROM_vep = paste0('chr', str_replace(str_replace(CHROM, 'XY', 'X'), 'MT', 'M'))) %>%
left_join(
    vep_df %>% rename('CHROM_vep'='CHROM'),
    by=c('CHROM_vep', 'POS', 'REF', 'ALT')
) %>%
left_join(csq_rules_df, by='Consequence') %>%
arrange(pvar_order) %>%
select(-CHROM_vep, -FILTER, -pvar_order) -> df

In [70]:
df %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(vep_csq_f, sep='\t', na = "NA", quote=F)


In [72]:
df %>% count(Csq) %>% pull(Csq)

[1] "intron" "others" "pav"    "pcv"    "ptv"    "utr"